In [1]:
from project_helpers import n4j_driver as n4j

In [4]:
%%time
with n4j.session() as session:
    random_users = session.run(
        '''
        MATCH (u:RealUser)
        RETURN u.user_id as user_id
        ORDER BY rand()
        LIMIT $lim
        ''',
        lim=1
    )
    for user in random_users:
        print(user['user_id'])
        similar_users = session.run(
            '''
            MATCH (otherUsers:RealUser) WHERE rand() < $percent
            WITH otherUsers
            MATCH (u:RealUser {user_id: $user_id})-[r1:rated]->(m:Movie),
                  (otherUsers)-[r2:rated]->(m)
            RETURN otherUsers.user_id AS uid,
                   sqrt(sum((r1.rating - r2.rating)^2)) AS distance,
                   count(r2) AS common_movies
            ORDER BY common_movies DESC, distance
            LIMIT $lim
            ''',
            percent=0.2, user_id=user['user_id'], lim=10
        )
        for similar_user in similar_users:
            print(similar_user)
        

1170975
<Record uid=387418 distance=15.033296378372908 common_movies=59>
<Record uid=2118461 distance=9.055385138137417 common_movies=58>
<Record uid=305344 distance=14.177446878757825 common_movies=58>
<Record uid=2439493 distance=18.33030277982336 common_movies=58>
<Record uid=1314869 distance=10.04987562112089 common_movies=57>
<Record uid=1664010 distance=12.649110640673518 common_movies=56>
<Record uid=2433610 distance=10.246950765959598 common_movies=54>
<Record uid=1403217 distance=11.789826122551595 common_movies=54>
<Record uid=2238060 distance=12.288205727444508 common_movies=54>
<Record uid=603543 distance=9.0 common_movies=53>
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 20.7 s
